# Import dependencies

In [0]:
import pyspark.sql.functions as F
import pandas as pd
from pyspark.sql.types import *
from pyspark.sql.functions import * 
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

In [0]:
dbutils.fs.ls('/mnt/your_mount_point')

#Data Reading

###Data Reading Json

In [0]:
df_json = spark.read.format('json').option('inferSchema',True)\
                    .option('header',True)\
                    .option('multiLine',False)\
                    .load('/FileStore/tables/drivers.json')

In [0]:
df_json.display()

## Reading from local csv file 

In [0]:
df = spark.read \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .csv("/Users/gmusabyimana/Downloads/PySpark-Full-Course-main/PySpark-Full-Course-main/DATA and NOTEBOOK/BigMart Sales.csv")

In [0]:
df.show()

## Move table from one location to another

In [0]:
%sql
CREATE TABLE bigmart_sale
USING DELTA
LOCATION '/mnt/default/Basics/delta'
AS
SELECT * FROM default.BigMart_sales;


## Data Reading Utils

In [0]:
dbutils.fs.ls('/FileStore/tables/')

In [0]:
df = spark.read.format('csv').option('inferSchema',True).option('header',True).load('/FileStore/tables/BigMart_Sales.csv')

- Creation of new source by using the sql query because the trial databricks platform does not allow to load data from extern source

In [0]:
df = spark.read.table('default.bigmart_sales')

In [0]:
display(df)


##Schema Definition

In [0]:
df.printSchema()

## DDL SCHEMA

In [0]:
my_ddl_schema = '''
                    Item_Identifier STRING,
                    Item_Weight STRING,
                    Item_Fat_Content STRING, 
                    Item_Visibility DOUBLE,
                    Item_Type STRING,
                    Item_MRP DOUBLE,
                    Outlet_Identifier STRING,
                    Outlet_Establishment_Year INT,
                    Outlet_Size STRING,
                    Outlet_Location_Type STRING, 
                    Outlet_Type STRING,
                    Item_Outlet_Sales DOUBLE 

                ''' 

In [0]:
df = spark.read.format('csv')\
            .schema(my_ddl_schema)\
            .option('header',True)\
            .load('/FileStore/tables/BigMart_Sales.csv') 

## StructType() SChema

This is used to enforce the schema of the dataframe before  you got the save data to the Silver Layer

In [0]:
my_strct_schema = StructType([
                                StructField('Item_Identifier',StringType(),True),
                                StructField('Item_Weight',StringType(),True),
                                StructField('Item_Fat_Content',StringType(),True),
                                StructField('Item_Visibility',StringType(),True),
                                StructField('Item_MRP',StringType(),True),
                                StructField('Outlet_Identifier',StringType(),True),
                                StructField('Outlet_Establishment_Year',StringType(),True),
                                StructField('Outlet_Size',StringType(),True),
                                StructField('Outlet_Location_Type',StringType(),True),
                                StructField('Outlet_Type',StringType(),True),
                                StructField('Item_Outlet_Sales',StringType(),True)

])

In [0]:
df = spark.read.format('csv')\
            .schema(my_strct_schema)\
            .option('header',True)\
            .load('/FileStore/tables/BigMart_Sales.csv')

In [0]:
df.printSchema()

# TRANSFORMATIONS

### Select statment

In [0]:
df.select(col('Item_Identifier'),col('Item_Weight'),col('Item_Fat_Content')).display()

### Alias  it can used also to rename column

In [0]:
df.select(col('Item_Identifier').alias('Item_ID')).display()

### Filter  rows base to a particular condition

In [0]:
df.filter(col('Item_Fat_Content')=='Regular').display()

In [0]:
## filter with many condition
df.filter((col('Item_Type') == 'Soft Drinks') & (col('Item_Weight')<10)).display()  

###Scenario 3 select data based on multiple condition

In [0]:
df.filter((col('Outlet_Size').isNull()) & (col('Outlet_Location_Type').isin('Tier 1','Tier 2'))).display()

## WithColumnRenamed

In [0]:
df.withColumnRenamed('Item_Weight','Item_Wt').display()

### WithColumn , we can use it to create new column   with static value or to store a result of transformation 

withColumn vs select

Use withColumn when:

Adding/replacing one or two columns

Logic is easier to read step-by-step

Use select when:

Transforming many columns

You want tight control over schema

Performance & clarity matte

In [0]:
df = df.withColumn('flag',lit("new")) 

In [0]:
display(df)

### create the new column in which you want to add a result of a functin or a computed value

In [0]:
df.withColumn('multiply',col('Item_Weight')*col('Item_MRP')).display()

###Scenario -2 use withColumn to apply a fonction on the specific column

In [0]:
df = df.withColumn('Item_Fat_Content',regexp_replace(col('Item_Fat_Content'),"Regular","Reg"))\
    .withColumn('Item_Fat_Content',regexp_replace(col('Item_Fat_Content'),"Low Fat","Lf"))

df.display()

### Apply the function with a specific rules this is common on the transformation

In [0]:
def appliquer_logique_nom(df):
    return df.withColumn(
        "nom_final",
        F.when(
            F.col("Item_Type").isNotNull() & (F.trim(F.col("Outlet_Type")) != ""),
            F.col("tem_Type")
        ).when(
            F.col("Outlet_Type").isNotNull() & (F.trim(F.col("Item_Type")).isNull()),
            F.col("Outlet_Type")
        ).when(
            F.col("Item_Type").isNull() & (F.trim(F.col("Outlet_Type")) == "R"),
            F.col("Return")
        ).otherwise(F.lit("INCONNU"))
    )


In [0]:
df = appliquer_logique_nom(df)
df.show()


### Type Casting

#### In PySpark, cast is how you change a column’s data type. You’ll use it all the time when cleaning or preparing data

In [0]:
df = df.withColumn('Item_Weight', col('Item_Weight').cast(StringType())) 

In [0]:
display(df)

In [0]:
df.printSchema()

### Sort

In [0]:
df.sort(col('Item_Weight').desc()).display()

In [0]:
df.sort(col('Item_Visibility').asc()).display()

### Scenario 2

In [0]:
df.sort(['Item_Weight','Item_Visibility'],ascending = [0,0]).display()

In [0]:
df.sort(['Item_weight','Item_Visibility'], ascending = [0,1]).display()

### Limit function is always used when you want to see onlu the sample of the data 

In [0]:
df.limit(10).display() 

### Drop

In [0]:
# drop one column
df.drop('Item_Visibility').display()
# drop multiple columns
df.drop('Item_Visibility','Item_Type').display()


## Drop Duplicates functions

**Why engineers still care**

distinct() signals set semantics

dropDuplicates() signals data quality cleanup

Usage convention

Use distinct() → analytical queries

Use dropDuplicates() → pipeline cleansing

In [0]:
df.drop_duplicates().display()
df.drop_duplicates(['Item_Type']).display()
df.drop_duplicates(['Item_Type','Item_MRP']).display()
df.drop_duplicates(['Item_Type','Item_MRP'],keep='last').display()

In [0]:
## this functions drop duplicates in the entire dataframe , is like dropDuplicate without any parameter
df.distinct().display()

In [0]:
window = Window.partitionBy("Outlet_Size").orderBy(col("Item_Type").desc())

df_dedup = (
    df
    .withColumn("rn", row_number().over(window))
    .filter(col("rn") == 1)
    .drop("rn")
)


In [0]:
df_dedup.display()

### Window Function

**Rule of thumb**

## Scenario	Best tool
- Bronze cleanup	dropDuplicates()
- Silver curation	Window + row_number
- Gold / serving	Delta MERGE
- Streaming	watermark + dropDuplicates

# Decision cheat sheet 🧠
## Requirement	Use this
- Remove exact row duplicates	dropDuplicates()
- Dedup on subset (don’t care which row)	dropDuplicates(cols)
- Keep latest / highest priority row	Window + row_number()
- Streaming dedup	withWatermark + dropDuplicates
- Incremental upsert	Delta MERGE

### UNION and UNION BY NAME

#### Preparing DataFrame

In [0]:
data1 = [('1','kad'),
        ('2','sid')]
schema1 = 'id STRING, name STRING' 

df1 = spark.createDataFrame(data1,schema1)

data2 = [('3','rahul'),
        ('4','jas')]
schema2 = 'id STRING, name STRING' 

df2 = spark.createDataFrame(data2,schema2)

In [0]:
df1.display()

In [0]:
df2.display()

### union of two dataframe

########  Use union() when you want to merge two tables with the same column count, regardless of column names.

######## Use unionByName() when you want to merge tables based on column names, ensuring that columns are matched and appended accordingly

In [0]:
data1 = [('kad','1',),
        ('sid','2',)]
schema1 = 'name STRING, id STRING' 

df1 = spark.createDataFrame(data1,schema1)

df1.display()

In [0]:
df1.union(df2).display()

### Union by Name

In [0]:
df1.unionByName(df2).display()

## String Functions

# Where string functions fit in big data pipelines

In real Spark jobs, string functions are mainly used for:

- Data cleansing → trimming spaces, fixing case, removing junk
- Standardization → consistent formats for joins & aggregations
- Parsing → extracting fields from semi-structured text
- Feature engineering → text-based columns for ML
- Validation → checking length, patterns, nulls

| Function    | Purpose                          | Example                 |
| ----------- | -------------------------------- | ----------------------- |
| `lower()`   | Convert to lowercase             | `lower(col("email"))`   |
| `upper()`   | Convert to uppercase             | `upper(col("country"))` |
| `initcap()` | Capitalize each word             | `initcap(col("name"))`  |
| `trim()`    | Remove leading & trailing spaces | `trim(col("name"))`     |
| `ltrim()`   | Remove left spaces               | `ltrim(col("name"))`    |
| `rtrim()`   | Remove right spaces              | `rtrim(col("name"))`    |


#### length function

In [0]:
df= df.withColumn("Outlet_lenght", length(col("Outlet_Type")))
display.df()


### lower() / upper() / initcap()

**Real-world example**
Whitespace issues are extremely common in ingestion from legacy systems.

In [0]:
df.select(upper('Item_Type').alias('upper_Item_Type')).display()

In [0]:
df.select(initcap('Item_Type').alias('upper_Item_Type')).display()

### Date Functions

# Scenario A: Extracting parts of a date
Use: year(), month(), dayofmonth(), dayofweek()

## Best Practices

- Always keep dates as DateType or TimestampType internally. Convert to string only for display.
- Reduces parsing overhead and allows proper date arithmetic.
- Use functions from pyspark.sql.functions instead of Python datetime when working on large datasets, because Spark functions run in parallel on the cluster.
- Be careful with formats in to_date() and date_format(). A mismatch can lead to nulls or wrong results.
- Use trunc() and next_day() for period-based analytics:

In [0]:
df = df.withColumn('curr_date',current_date())

df.display()

In [0]:
df = df.withColumn('week_after',date_add('curr_date',7))

df.display()

#### Date_Sub()

In [0]:
df.withColumn('week_before',date_sub('curr_date',7)).display()

In [0]:
df = df.withColumn('week_before',date_add('curr_date',-7)) 

df.display()

### DateDIFF

In [0]:
df = df.withColumn('datediff',datediff('week_after','curr_date'))

df.display()

### Date_Format()

In [0]:
df = df.withColumn('week_before',date_format('week_before','dd-MM-yyyy'))

df.display()

In [0]:
df_extract = df.groupBy(year('week_before'), month('week_before')).count()
df_extract.display()

### Handling Nulls

#### Dropping NUlls

# Real-world Data Engineer Scenarios
Scenario	Code Example	Notes
- Remove rows with any null in critical columns	df.dropna(subset=["user_id", "transaction_id"])	Ensures primary keys are always valid
- Keep rows with at least 3 valid fields in wide table	df.dropna(thresh=3)	Useful for surveys/logs with optional columns
- Remove rows where all columns are null	df.dropna(how='all')	Avoids meaningless empty rows in logs
- Clean data before writing to Delta Lake	df.dropna(subset=["user_id", "event_time"]).write.format("delta").save("/mnt/delta/events")	Ensures downstream tables are clean
- Combine dropna with fillna	df.dropna(subset=["id"]).fillna({"age": 0})	Critical for ETL: remove rows with no id but fill missing age

In [0]:
df.dropna('all').display()

In [0]:
df.dropna('any').display()

In [0]:
df.dropna(subset=['Outlet_Size']).display()

**Summary**

- dropna() is a powerful tool to clean nulls in PySpark
- Key parameters: how, subset, thresh
- Data Engineers should always consider performance and column relevance
- Often used before joins, aggregations, and writes to storage

- dropna() ensures data integrity
- fillna() handles optional fields
- dropDuplicates() ensures uniqueness

#### Filling Nulls

In [0]:
df.fillna('NotAvailable').display()

In [0]:
df.fillna('NotAvailable',subset=['Outlet_Size']).display()

### SPLIT and Indexing

#### SPLIT

In [0]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')).display()

#### Indexing

In [0]:
df.withColumn('Outlet_Type',split('Outlet_Type',' ')[1]).display()

### Explode

In [0]:
df_exp = df.withColumn('Outlet_Type',split('Outlet_Type',' '))

df_exp.display()

In [0]:
df_exp.withColumn('Outlet_Type',explode('Outlet_Type')).display()

In [0]:
df_exp.display()

In [0]:
df_exp.withColumn('Type1_flag',array_contains('Outlet_Type','Type1')).display()

### GroupBY

### Scenario_O1

In [0]:
df.groupBy('Item_Type').agg(sum('Item_MRP')).display()

### Scenario 2 

In [0]:
df.groupBy('Item_Type').agg(avg('Item_MRP')).display()

In [0]:
df.groupBy('Item_Type','Outlet_Size').agg(sum('Item_MRP').alias('Total_MRP')).display()

In [0]:
df.groupBy('Item_Type','Outlet_Size').agg(sum('Item_MRP'),avg('Item_MRP')).display()

### Collect_List
- collect_list() is an aggregation function in PySpark (from pyspark.sql.functions) that:
- Collects values from multiple rows into a single array (list) per group.
- It is typically used with groupBy().

## **Performance Considerations (Expert Level)**

- In large U.S. enterprise data pipelines:
- collect_list causes shuffle
- Large groups → skew problems
- Can increase memory usage
### - Best Practices:
- Repartition properly
- Handle data skew
- Use broadcast joins where applicable 
- Consider window functions if better suited

In [0]:
data = [('user1','book1'),
        ('user1','book2'),
        ('user2','book2'),
        ('user2','book4'),
        ('user3','book1')]

schema = 'user string, book string'

df_book = spark.createDataFrame(data,schema)

df_book.display()

In [0]:
df_book.groupBy('user').agg(collect_list('book')).display()

In [0]:
df.select('Item_Type','Outlet_Size','Item_MRP').display()

## PIVOT

In [0]:
df.groupBy('Item_Type').pivot('Outlet_Size').agg(avg('Item_MRP')).display()

### When-Otherwise

“I translate business requirements into boolean column expressions using when-otherwise, ensure proper ordering and null handling, avoid UDFs for performance, and externalize complex rule sets into configuration tables to maintain scalability and maintainability in Databricks production pipelines.”
Business rules are:

Conditions defined by business stakeholders

- Policy-based logic
- Compliance rules
- Risk scoring logic
- Classification logic
- Data standardization logi

In [0]:
df = df.withColumn('veg_flag',when(col('Item_Type')=='Meat','Non-Veg').otherwise('Veg'))
display(df)

In [0]:
##from pyspark.sql.functions import expr

df_BusinessRules = df.withColumn(
    "risk_flag",
    expr("""
        CASE 
            WHEN Item_Outlet_Sales> 10000 AND veg_flag = 'Veg' THEN 'High'
            WHEN Outlet_Type != 'Supermarket Type1' THEN 'High'
            WHEN Item_Outlet_Sales > 5000 THEN 'Medium'
            ELSE 'Low'
        END
    """)
)
display(df_BusinessRules)


In [0]:
df.withColumn('veg_exp_flag',when(((col('veg_flag')=='Veg') & (col('Item_MRP')<100)),'Veg_Inexpensive')\
                            .when((col('veg_flag')=='Veg') & (col('Item_MRP')>100),'Veg_Expensive')\
                            .otherwise('Non_Veg')).display()

### JOINS

In [0]:
dataj1 = [('1','gaur','d01'),
          ('2','kit','d02'),
          ('3','sam','d03'),
          ('4','tim','d03'),
          ('5','aman','d05'),
          ('6','nad','d06')] 

schemaj1 = 'emp_id STRING, emp_name STRING, dept_id STRING' 

df1 = spark.createDataFrame(dataj1,schemaj1)

dataj2 = [('d01','HR'),
          ('d02','Marketing'),
          ('d03','Accounts'),
          ('d04','IT'),
          ('d05','Finance')]

schemaj2 = 'dept_id STRING, department STRING'

df2 = spark.createDataFrame(dataj2,schemaj2)

In [0]:
df1.join(df2, df1['dept_id']==df2['dept_id'],'inner').display()
df1.join(df2,df1['dept_id']==df2['dept_id'],'left').display()
df1.join(df2,df1['dept_id']==df2['dept_id'],'right').display()
df1.join(df2,df1['dept_id']==df2['dept_id'],'anti').display()

## Window Functions

Row_Number()

In [0]:
df.withColumn('rowCol',row_number().over(Window.orderBy('Item_Identifier'))).display()

In [0]:
windowSpec = Window.partitionBy("Item_Fat_Content").orderBy("Item_MRP")

df = df.withColumn("rank", row_number().over(windowSpec))
display(df)


#### RANK VS DENSE RANK 

In [0]:
df.withColumn('rank',rank().over(Window.orderBy(col('Item_Identifier').desc())))\
        .withColumn('denseRank',dense_rank().over(Window.orderBy(col('Item_Identifier').desc()))).display()

In [0]:
df.withColumn('dum',sum('Item_MRP').over(Window.orderBy('Item_Identifier').rowsBetween(Window.unboundedPreceding,Window.currentRow))).display()

#### Cumulative Sum

In [0]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type'))).display()

In [0]:
df.withColumn('cumsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.currentRow))).display()

In [0]:
df.withColumn('totalsum',sum('Item_MRP').over(Window.orderBy('Item_Type').rowsBetween(Window.unboundedPreceding,Window.unboundedFollowing))).display()

### USER DEFINED FUNCTIONS (UDF)

Data Standardization

- Normalize phone numbers
- Clean messy strings
- Custom date parsing
- Complex regex cleanup

In [0]:
def my_func(x):
    return x*x 

In [0]:
my_udf = udf(my_func)

In [0]:
df.withColumn('mynewcol',my_udf('Item_MRP')).display()

### DATA WRITING

In [0]:
df.write.format('csv')\
        .save('/FileStore/tables/CSV/data.csv')

### Append Mode

In [0]:
df.write.format('csv')\
        .mode('append')\
        .save('/FileStore/tables/CSV/data.csv')

In [0]:
        df.write.format('csv')\
        .mode('append')\
        .option('path','/FileStore/tables/CSV/data.csv')\
        .save()

## Overwrite Mode

In [0]:
df.write.format('csv')\
.mode('overwrite')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

In [0]:
# error
df.write.format('csv')\
.mode('error')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

In [0]:
# Ingore
df.write.format('csv')\
.mode('ignore')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

## Parquet

In [0]:
df.write.format('parquet')\
.mode('overwrite')\
.option('path','/FileStore/tables/CSV/data.csv')\
.save()

## Table 

In [0]:
df.write.format('parquet')\
.mode('overwrite')\
.saveAsTable('my_table')